In [1]:
import pycountry
import pandas as pd
import numpy as np
import astropy.units as u
from xlsx2csv import Xlsx2csv

In [2]:
# Needed for pycountry to work correctly
def get_country(country_name):
    if country_name == "Russia":
        return pycountry.countries.get(alpha_2="RU")
    if country_name == "Turkey":
        return pycountry.countries.get(alpha_2="TR")
    if country_name == "South Korea":
        return pycountry.countries.get(alpha_2="KR")
    if country_name == "Taiwan":
        return pycountry.countries.get(alpha_2="TW")
    return pycountry.countries.get(name=country_name)

In [4]:
## Source: https://www.top500.org/lists/green500/2024/06/
Xlsx2csv("./data/green500_top_202406.xlsx", outputencoding="utf-8").convert("./data/green500_top_202406.csv")
green500 = pd.read_csv('./data/green500_top_202406.csv')

## Source: https://ourworldindata.org/grapher/carbon-intensity-electricity
# License: OurWorldInData.org/energy | CC BY [https://creativecommons.org/licenses/by/4.0/]
carbon_intensities = pd.read_csv('./data/carbon-intensity-electricity.csv')

green500["Carbon footprint [kgCO2e / PFlops-day]"] = -1.0
for index, row in green500.iterrows():
    system_name = row['Name']
    if system_name == "nan":
        print(row['Site'])
        green500.at[index, "Name"] = row['Site']
    country_name = row['Country']

    if np.isnan(row['Energy Efficiency [GFlops/Watts]']):
        green500 = green500.drop(index)
        continue
    # Drop machines older than year X
    # if row['Year'] < X:
    #     green500 = green500.drop(index)
    #     continue
    country = get_country(country_name)
    carbon_intensity = carbon_intensities[carbon_intensities['Code']==country.alpha_3]['Carbon intensity of electricity - gCO2/kWh'].values[0] * u.g / (u.kW * u.h)
    energy_efficiency = row['Energy Efficiency [GFlops/Watts]'] * (u.GB/u.s) /u.Watt
    carbon_footprint =  (carbon_intensity / energy_efficiency).to('kg Pbyte-1 s day-1').value #   [ kgCO2e / PFlops-day ]
    green500.at[index, "Carbon footprint [kgCO2e / PFlops-day]"]  = round(carbon_footprint, 1)


climate500 = green500.sort_values(by=["Carbon footprint [kgCO2e / PFlops-day]"])

climate500.to_csv("./data/climate500.csv")
